<a href="https://colab.research.google.com/github/balakumar-dataengineer/testrepo/blob/master/Pyspark_From_Json_to_Map_Type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('Pyspark') \
                          .master('local') \
                          .getOrCreate()

print(spark)

In [12]:
data=[
    (1,'bala','{"hair":"black","eye":"black"}'),
    (2,'kumar','{"hair":"blue","eye":"brown"}')
]

schema=['id','name','properties']

df=spark.createDataFrame(data,schema)
df.show(truncate=False)
df.printSchema()

+---+-----+------------------------------+
|id |name |properties                    |
+---+-----+------------------------------+
|1  |bala |{"hair":"black","eye":"black"}|
|2  |kumar|{"hair":"blue","eye":"brown"} |
+---+-----+------------------------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- properties: string (nullable = true)



In [30]:
from pyspark.sql.functions import from_json,explode
from pyspark.sql.types import MapType,StringType,StructType,StructField

df1=df.select('id','name','properties',from_json(df.properties,MapType(StringType(),StringType())).alias('propMap'))

df1.select('id','name','properties','propMap',explode('propMap').alias('key','value'),df1.propMap.hair.alias('keypair'),df1.propMap.eye.alias('eyepair')) \
   .show(truncate=False)

+---+-----+------------------------------+-----------------------------+----+-----+-------+-------+
|id |name |properties                    |propMap                      |key |value|keypair|eyepair|
+---+-----+------------------------------+-----------------------------+----+-----+-------+-------+
|1  |bala |{"hair":"black","eye":"black"}|{hair -> black, eye -> black}|hair|black|black  |black  |
|1  |bala |{"hair":"black","eye":"black"}|{hair -> black, eye -> black}|eye |black|black  |black  |
|2  |kumar|{"hair":"blue","eye":"brown"} |{hair -> blue, eye -> brown} |hair|blue |blue   |brown  |
|2  |kumar|{"hair":"blue","eye":"brown"} |{hair -> blue, eye -> brown} |eye |brown|blue   |brown  |
+---+-----+------------------------------+-----------------------------+----+-----+-------+-------+



In [28]:
df.show(truncate=False)

+---+-----+------------------------------+
|id |name |properties                    |
+---+-----+------------------------------+
|1  |bala |{"hair":"black","eye":"black"}|
|2  |kumar|{"hair":"blue","eye":"brown"} |
+---+-----+------------------------------+



In [38]:
schema1=StructType(
    [StructField('hair',StringType(),True),
    StructField('eye',StringType(),True)]
)

df2=df.select('id','name','properties',from_json(df.properties,schema1).alias('propStruct'))

In [45]:
df2.select('id','name','properties','propStruct.*').show(truncate=False)

df2.printSchema()

+---+-----+------------------------------+-----+-----+
|id |name |properties                    |hair |eye  |
+---+-----+------------------------------+-----+-----+
|1  |bala |{"hair":"black","eye":"black"}|black|black|
|2  |kumar|{"hair":"blue","eye":"brown"} |blue |brown|
+---+-----+------------------------------+-----+-----+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- properties: string (nullable = true)
 |-- propStruct: struct (nullable = true)
 |    |-- hair: string (nullable = true)
 |    |-- eye: string (nullable = true)

